In [1]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered


In [2]:
import pandas as pd
import nilearn
import numpy as np
import glob 
import os
import pickle

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf


from pathlib import Path


from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

import seaborn as sns


from nltools.data import Brain_Data, Adjacency
from nltools.mask import roi_to_brain, expand_mask
from nltools.stats import fdr, threshold

%matplotlib inline


/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
def onetoughjar(path2dic):
    with open(path2dic, 'rb') as pickle_file:
        try:
            while True:
                output = pickle.load(pickle_file)
        except EOFError:
            pass
    return(output)

In [4]:
def interactor(m):
    X['int'] = X['demo_comb_income_v2']*X['%s'%m]
    # Fit the multinomial logistic regression model
    logit_model = sm.Logit(y, X)
    result = logit_model.fit()
    return(result)



In [ ]:
savedDict = onetoughjar('/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024/tmp/SSBABCDanalysis_11-23-2024_07-36-16')

In [ ]:
savedDict.keys()

In [ ]:
savedDict['logistic'].params

In [ ]:
colz = savedDict['X_reg']

In [ ]:
list(colz.columns[1:])

In [ ]:
X = savedDict['X']

In [ ]:
X = X[list(colz.columns[1:])]

In [ ]:
X.shape

In [ ]:
y = savedDict['y']

In [ ]:
y.shape

In [ ]:
brain = ['rsfmri_c_ngd_ad_ngd_cgc', 'rsfmri_c_ngd_ad_ngd_vs',
       'rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_c_ngd_smh_ngd_smm',
       'rsfmri_c_ngd_sa_ngd_vta', 'rsfmri_cor_ngd_au_scs_aalh',
       'rsfmri_cor_ngd_au_scs_ptrh', 'rsfmri_cor_ngd_au_scs_vtdcrh',
       'rsfmri_cor_ngd_cerc_scs_cdelh', 'rsfmri_cor_ngd_cerc_scs_ptlh',
       'rsfmri_cor_ngd_cerc_scs_aglh', 'rsfmri_cor_ngd_cerc_scs_vtdclh',
       'rsfmri_cor_ngd_cerc_scs_hprh', 'rsfmri_cor_ngd_cerc_scs_agrh',
       'rsfmri_cor_ngd_cerc_scs_vtdcrh', 'rsfmri_cor_ngd_copa_scs_thplh',
       'rsfmri_cor_ngd_copa_scs_vtdclh', 'rsfmri_cor_ngd_copa_scs_plrh',
       'rsfmri_cor_ngd_df_scs_ptlh', 'rsfmri_cor_ngd_df_scs_pllh',
       'rsfmri_cor_ngd_df_scs_aalh', 'rsfmri_cor_ngd_none_scs_ptlh',
       'rsfmri_cor_ngd_none_scs_thprh', 'rsfmri_cor_ngd_none_scs_cderh',
       'rsfmri_cor_ngd_rst_scs_bs', 'rsfmri_cor_ngd_rst_scs_thprh',
       'rsfmri_cor_ngd_rst_scs_hprh', 'rsfmri_cor_ngd_rst_scs_vtdcrh',
       'rsfmri_cor_ngd_smh_scs_crcxlh', 'rsfmri_cor_ngd_smh_scs_pllh',
       'rsfmri_cor_ngd_smh_scs_cderh', 'rsfmri_cor_ngd_smh_scs_ptrh',
       'rsfmri_cor_ngd_smh_scs_hprh', 'rsfmri_cor_ngd_smh_scs_aarh',
       'rsfmri_cor_ngd_smm_scs_hplh', 'rsfmri_cor_ngd_smm_scs_aglh',
       'rsfmri_cor_ngd_smm_scs_thprh', 'rsfmri_cor_ngd_sa_scs_crcxlh',
       'rsfmri_cor_ngd_sa_scs_thplh', 'rsfmri_cor_ngd_vta_scs_hplh',
       'rsfmri_cor_ngd_vs_scs_hplh', 'rsfmri_cor_ngd_vs_scs_thprh',
       'rsfmri_cor_ngd_vs_scs_plrh', 'rsfmri_cor_ngd_vs_scs_hprh', 'interview_age',
       'bmi_perc', 'sex_M']

In [ ]:
pvalues = {}

In [ ]:
for ROI in brain:
    try:
        pvalues[ROI] = interactor(ROI)
    except:
        pvalues[ROI] = 'error'
    

In [ ]:
intP = []
for ps in pvalues.values():
    intP.append(ps.pvalues[-1])

In [ ]:
# Create a Series without a custom index
seriesP = pd.Series(intP)

# Assign a custom index
seriesP.index = pvalues.keys()
# Replace 'r' with 1
seriesP = seriesP.replace('r', 1)

# Replace NaN with 1
seriesP = seriesP.fillna(1)

# Display the Series
print(seriesP)

In [ ]:
# Perform FDR correction
rejected, pvals_corrected, _, _ = multipletests(seriesP, alpha=0.05, method='fdr_bh')

# Print the results
print("Rejected (Significant after FDR):", rejected)
print("Corrected p-values:", pvals_corrected)


In [ ]:
# Create a Series without a custom index
sigP = pd.Series(pvals_corrected)
# Assign a custom index
sigP.index = seriesP.index


In [ ]:
# Filter the series using the boolean list
filtered_values = sigP[rejected]

In [ ]:
filtered_values.index

# Summary
Household income interacts with:
* rsfmri_c_ngd_cgc_ngd_cgc         1.640322e-02 Average correlation between cingulo-opercular network and cingulo-opercular network
* rsfmri_cor_ngd_df_scs_pllh       2.499484e-02 Average correlation between default network and ASEG ROI left-pallidum
* rsfmri_cor_ngd_none_scs_cderh    1.640322e-02 Average correlation between none network and ASEG ROI right-caudate
* rsfmri_cor_ngd_sa_scs_thplh      3.252892e-02 Average correlation between salience network and ASEG ROI left-thalamus-proper
* interview_age                    1.548630e-06
* bmi_perc                         1.044164e-13

# Probing the interaction

In [ ]:
result = pvalues['rsfmri_c_ngd_cgc_ngd_cgc']

In [ ]:
def tableMaker(result):
    # Assuming `result` is your BinaryResultsWrapper object
    # Extract coefficients, confidence intervals, and compute odds ratios
    params = result.params
    conf = result.conf_int()
    conf['OR'] = np.exp(params)  # Compute odds ratios
    conf.columns = ['2.5%', '97.5%', 'OR']  # Rename columns

    # Compute odds ratios for confidence intervals
    conf['2.5%'] = np.exp(conf['2.5%'])
    conf['97.5%'] = np.exp(conf['97.5%'])

    # Format as a DataFrame for publication
    table = conf[['OR', '2.5%', '97.5%']]

    # Optionally, round the table for readability
    table = table.round(2)
    return(table)

    

In [ ]:
tableMaker(savedDict['logistic'])

In [ ]:
tableHolder = {}

In [ ]:
for ROI in list(filtered_values.index):
    result = pvalues['%s'%ROI]
    tableHolder['%s'%ROI] = tableMaker(result)

In [ ]:
tableHolder.keys()

In [ ]:
tableHolder['bmi_perc']

In [ ]:
savedDict.update({'sigP':sigP})

In [ ]:
savedDict.update({'odds':tableHolder})

In [ ]:
savedDict['odds']

In [ ]:
import pickle
from datetime import datetime
from time import time


In [ ]:
datefmt='%m-%d-%Y_%I-%M-%S'

In [ ]:
def adillyofapickle(basepath,dic, name):
    st = datetime.fromtimestamp(time()).strftime(datefmt)
    if os.path.exists(os.path.join(basepath,'tmp')):
        print('already have tmp')
    else:
        os.makedirs(os.path.join(basepath,'tmp'))
    pickle.dump(dic, open(os.path.join(basepath,'tmp','%s_%s'%(name,st)), 'wb'), protocol=4)

In [ ]:
basepath = '/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024'




In [ ]:
adillyofapickle(basepath,savedDict, 'SSBABCDanalysisInt')

## Probe the interaction for plots
Didn't have the dataset with factor level of CHI, loaded that and re-merged with original

In [5]:
dataPath = '/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024/tmp'
savedDict = onetoughjar(os.path.join(dataPath,'SSBABCDanalysisInt_11-26-2024_10-44-32'))

In [6]:

factorDF = pd.read_csv('/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024/data/justCHI.csv')

In [7]:
df = pd.merge(factorDF, savedDict['df'], on='subjectkey', how='outer')

In [10]:
df.shape

(3700, 442)

In [9]:
df = df.dropna(subset=['demo_comb_income_factor'])

In [11]:
nonBrain = df[['demo_comb_income_factor','interview_age', 'bmi_perc',  'sex_M','ssb_group',]]

In [22]:
ROIs = list(savedDict['X_reg'].columns[1:-5])

In [26]:
brain = df[ROIs]

In [ ]:
df.shape

In [27]:
Xfact = brain.join(nonBrain)

In [29]:
Xfact['const'] = 1

In [31]:
Xfact.shape

(3700, 50)

In [32]:
CHIlevels = [1,2,3,4,5,6,7,8,9,10]

In [33]:
breakOut = {}

In [34]:
for i in CHIlevels:
    print(i)
    subset_df_ = Xfact[Xfact['demo_comb_income_factor'] == i]
    subset_df_ = subset_df_.drop('demo_comb_income_factor', axis=1)
    breakOut[i] = subset_df_


1
2
3
4
5
6
7
8
9
10


In [46]:
testers = ['rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_cor_ngd_df_scs_pllh',
       'rsfmri_cor_ngd_none_scs_cderh', 'rsfmri_cor_ngd_sa_scs_thplh',
       'interview_age', 'bmi_perc']

In [36]:
X = breakOut[8].drop('ssb_group', axis = 1)

In [37]:
y = breakOut[8]['ssb_group']

In [38]:
X.shape

(621, 48)

In [39]:
y.value_counts()

0.0    356
1.0    265
Name: ssb_group, dtype: int64

In [40]:
def modelerTest(i):
    X = breakOut[i].drop('ssb_group', axis = 1)
    y = breakOut[i]['ssb_group']
    logit_model = sm.Logit(y, X)
    result = logit_model.fit()
    return(result)

In [49]:
probInt = {}

In [53]:
for i in breakOut.keys():
    print(i)
    try:
        probInt[i] = modelerTest(i)
    except:  # Catch the LinAlgError properly
        probInt[i] = 'Check error'


1
         Current function value: inf
         Iterations: 35
2
3
4
5
6
Optimization terminated successfully.
         Current function value: 0.206592
         Iterations 10
7
Optimization terminated successfully.
         Current function value: 0.306252
         Iterations 8
8
Optimization terminated successfully.
         Current function value: 0.590394
         Iterations 6
9
Optimization terminated successfully.
         Current function value: 0.597549
         Iterations 5
10
Optimization terminated successfully.
         Current function value: 0.543546
         Iterations 6


/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [54]:
probInt

{1: 'Check error',
 2: 'Check error',
 3: 'Check error',
 4: 'Check error',
 5: 'Check error',
 6: <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fc190502f70>,
 7: <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fc190510220>,
 8: <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fc190510460>,
 9: <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fc1905107f0>,
 10: <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7fc190510b80>}

In [47]:
testers = ['rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_cor_ngd_df_scs_pllh',
           'rsfmri_cor_ngd_none_scs_cderh', 'rsfmri_cor_ngd_sa_scs_thplh',
           'interview_age', 'bmi_perc']

# Filter the p-values using the testers list
filtered_pvalues = test.pvalues[test.pvalues.index.isin(testers)]

print(filtered_pvalues)


rsfmri_c_ngd_cgc_ngd_cgc         8.893909e-01
rsfmri_cor_ngd_df_scs_pllh       1.819174e-01
rsfmri_cor_ngd_none_scs_cderh    3.381372e-02
rsfmri_cor_ngd_sa_scs_thplh      1.727194e-02
interview_age                    1.982785e-07
bmi_perc                         4.123034e-04
dtype: float64


In [59]:
for i in range(6,11):
    print(i)
    test = probInt[i]
    filtered_pvalues = round(test.pvalues[test.pvalues.index.isin(testers)],4)
    print(filtered_pvalues)


6
rsfmri_c_ngd_cgc_ngd_cgc         0.5101
rsfmri_cor_ngd_df_scs_pllh       0.3216
rsfmri_cor_ngd_none_scs_cderh    0.2169
rsfmri_cor_ngd_sa_scs_thplh      0.7944
interview_age                    0.0000
bmi_perc                         0.0000
dtype: float64
7
rsfmri_c_ngd_cgc_ngd_cgc         0.1608
rsfmri_cor_ngd_df_scs_pllh       0.1680
rsfmri_cor_ngd_none_scs_cderh    0.1748
rsfmri_cor_ngd_sa_scs_thplh      0.6229
interview_age                    0.0000
bmi_perc                         0.0000
dtype: float64
8
rsfmri_c_ngd_cgc_ngd_cgc         0.8894
rsfmri_cor_ngd_df_scs_pllh       0.1819
rsfmri_cor_ngd_none_scs_cderh    0.0338
rsfmri_cor_ngd_sa_scs_thplh      0.0173
interview_age                    0.0000
bmi_perc                         0.0004
dtype: float64
9
rsfmri_c_ngd_cgc_ngd_cgc         0.3018
rsfmri_cor_ngd_df_scs_pllh       0.9878
rsfmri_cor_ngd_none_scs_cderh    0.1449
rsfmri_cor_ngd_sa_scs_thplh      0.3834
interview_age                    0.5750
bmi_perc                   

In [ ]:
savedDict.keys()

In [60]:
Xfact.columns

Index(['rsfmri_c_ngd_ad_ngd_cgc', 'rsfmri_c_ngd_ad_ngd_vs',
       'rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_c_ngd_smh_ngd_smm',
       'rsfmri_c_ngd_sa_ngd_vta', 'rsfmri_cor_ngd_au_scs_aalh',
       'rsfmri_cor_ngd_au_scs_ptrh', 'rsfmri_cor_ngd_au_scs_vtdcrh',
       'rsfmri_cor_ngd_cerc_scs_cdelh', 'rsfmri_cor_ngd_cerc_scs_ptlh',
       'rsfmri_cor_ngd_cerc_scs_aglh', 'rsfmri_cor_ngd_cerc_scs_vtdclh',
       'rsfmri_cor_ngd_cerc_scs_hprh', 'rsfmri_cor_ngd_cerc_scs_agrh',
       'rsfmri_cor_ngd_cerc_scs_vtdcrh', 'rsfmri_cor_ngd_copa_scs_thplh',
       'rsfmri_cor_ngd_copa_scs_vtdclh', 'rsfmri_cor_ngd_copa_scs_plrh',
       'rsfmri_cor_ngd_df_scs_ptlh', 'rsfmri_cor_ngd_df_scs_pllh',
       'rsfmri_cor_ngd_df_scs_aalh', 'rsfmri_cor_ngd_none_scs_ptlh',
       'rsfmri_cor_ngd_none_scs_thprh', 'rsfmri_cor_ngd_none_scs_cderh',
       'rsfmri_cor_ngd_rst_scs_bs', 'rsfmri_cor_ngd_rst_scs_thprh',
       'rsfmri_cor_ngd_rst_scs_hprh', 'rsfmri_cor_ngd_rst_scs_vtdcrh',
       'rsfmri_cor_ngd_smh_scs_cr

In [62]:
savedDict.keys()

dict_keys(['df', 'X', 'y', 'X_train', 'X_test', 'y_train', 'y_test', 'best_pipeline', 'X_reg', 'X_trainReg', 'X_testReg', 'y_trainReg', 'y_testReg', 'logistic', 'sigP', 'odds'])

In [68]:
y_trainReg

NameError: name 'y_trainReg' is not defined

In [70]:
savedDict['odds'][]

{'rsfmri_c_ngd_cgc_ngd_cgc':                                    OR  2.5%  97.5%
 rsfmri_c_ngd_ad_ngd_cgc          0.82  0.11   6.39
 rsfmri_c_ngd_ad_ngd_vs           3.20  0.80  12.84
 rsfmri_c_ngd_cgc_ngd_cgc         0.50  0.11   2.30
 rsfmri_c_ngd_smh_ngd_smm         2.30  0.78   6.83
 rsfmri_c_ngd_sa_ngd_vta          1.08  0.29   4.07
 rsfmri_cor_ngd_au_scs_aalh       0.14  0.04   0.53
 rsfmri_cor_ngd_au_scs_ptrh       1.56  0.40   6.15
 rsfmri_cor_ngd_au_scs_vtdcrh     0.85  0.20   3.52
 rsfmri_cor_ngd_cerc_scs_cdelh    0.07  0.01   0.34
 rsfmri_cor_ngd_cerc_scs_ptlh     0.93  0.31   2.86
 rsfmri_cor_ngd_cerc_scs_aglh     0.71  0.10   4.99
 rsfmri_cor_ngd_cerc_scs_vtdclh   0.74  0.38   1.43
 rsfmri_cor_ngd_cerc_scs_hprh     1.42  0.20   9.80
 rsfmri_cor_ngd_cerc_scs_agrh     1.55  0.42   5.77
 rsfmri_cor_ngd_cerc_scs_vtdcrh   0.57  0.11   2.97
 rsfmri_cor_ngd_copa_scs_thplh    1.34  0.22   7.97
 rsfmri_cor_ngd_copa_scs_vtdclh   4.08  0.56  29.87
 rsfmri_cor_ngd_copa_scs_plrh     1.